# Chapter 4 파이토치 기본

## 4.1 텐서

4.1.1 여러 가지 텐서
- 파이토치의 기본 단위. GPU 연산을 가능하게 하며 Numpy의 배열과 유사

In [1]:
import torch
import numpy as np

x = torch.empty(5,4) # 빈 텐서 생성
print(x)

tensor([[4.7429e+30, 2.0539e-35, 4.1589e+21, 9.9504e+21],
        [4.1589e+21, 2.8295e+30, 1.8545e+25, 1.7539e+19],
        [1.6635e+22, 2.6375e+30, 1.1210e-43, 2.9217e-41],
        [1.6816e+20, 4.4254e+30, 1.8545e+25, 1.7539e+19],
        [1.6635e+22, 7.3680e+28, 4.4248e+30, 1.8545e+25]])


In [2]:
torch.ones(3,3) # 3x3 일 행렬

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [3]:
torch.zeros(2) # 2행 영 벡터

tensor([0., 0.])

In [4]:
torch.rand(5,6) # 5x6 무작위 행렬

tensor([[0.3971, 0.2219, 0.3922, 0.7164, 0.1959, 0.8042],
        [0.9737, 0.9750, 0.2905, 0.8645, 0.6756, 0.2383],
        [0.1369, 0.8651, 0.5220, 0.0152, 0.3336, 0.7998],
        [0.4045, 0.8090, 0.9492, 0.1974, 0.8041, 0.5510],
        [0.2327, 0.1791, 0.2983, 0.5620, 0.3520, 0.1191]])

### 4.1.2 리스트, 넘파이 배열을 텐서로 만들기

In [5]:
l = [13,4] # 리스트 생성
r = np.array([4,56,7]) # 넘파이 배열 생성
torch.tensor(l)

tensor([13,  4])

In [6]:
torch.tensor(r)

tensor([ 4, 56,  7])

### 4.1.3 텐서의 크기, 타입, 연산

In [7]:
# 텐서의 크기 확인

x.size()[1]

4

In [8]:
x = torch.rand(2,2) # 2x2 랜덤 행렬
y = torch.rand(2,2)
torch.add(x,y) # x+y
y.add(x) # y에 x를 더함

tensor([[1.9438, 0.1904],
        [0.7438, 1.3290]])

### 4.1.4 텐서의 크기 변환

In [9]:
x = torch.rand(8,8)
print(x.size())

torch.Size([8, 8])


In [10]:
a = x.view(64) # 크기를 바꿔주는 view 8x8 -> 64
print(a.size())

torch.Size([64])


In [11]:
b = x.view(-1,4,4) # -1은 원래 크기가 되게 하는 값 8x8 -> -1x4x4 즉, 4x4x4 배열
print(b.size())

torch.Size([4, 4, 4])


### 4.1.5 텐서에서 넘파이 배열로 변환

In [12]:
x = torch.rand(8,8)
y = x.numpy()
type(y)

numpy.ndarray

### 4.1.6 단일 텐서에서 값으로 반환하기

In [13]:
x = torch.ones(1)
print(x.item())

1.0
